In [1]:
#link drive to google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
cd /content

/content


In [2]:
import scipy.io as sio
import numpy as np
import csv
from pathlib import Path
import sklearn
import pandas as pd
import regex as re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from copy import copy

#download from nltk
nltk.download('punkt')
nltk.download('stopwords')
stop = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
!pip install git+https://github.com/guillaumegenthial/tf_metrics.git

  Cloning https://github.com/guillaumegenthial/tf_metrics.git to /tmp/pip-req-build-el3b8n9l
  Running setup.py bdist_wheel for tf-metrics ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-kuxximcf/wheels/da/6c/c8/663ef339a0666590dc53bd13bab86643a1f9c35b26742d7876
Successfully built tf-metrics


In [8]:
!git clone https://github.com/guillaumegenthial/tf_ner.git

Cloning into 'tf_ner'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 165 (delta 70), reused 143 (delta 51), pack-reused 0
Receiving objects: 100% (165/165), 126.19 KiB | 3.00 MiB/s, done.
Resolving deltas: 100% (70/70), done.


# **Preprocessing Ayurveda PDF Files**

In [0]:
#read text file for train data and create data file in which to write processed data
fread = open('test_data.txt')  
fwrite = open('test_paras.txt','w+')
lines = fread.readlines()

In [0]:
#each para is one document
paras=[]
para=""
for line in lines:
  line = line.strip()
  if line=='':
    paras.append(para) 
    para=""
  else:
    para=para+" "+line.strip()
paras.append(para)
print(len(paras))

In [0]:
#finish preprocessing
count=1
for line in paras:
    line = line.strip()
    line = line.lower()
    
    tokens = word_tokenize(line)

    tmp = tokens[:]
    #so w/w is removed. but problem with removal of brackets
    for token in tokens:
      if token in ['(','[',']',')']:
        continue
      elif not token.isalnum(): 
        tmp.remove(token)
    for n,t in enumerate(tokens):
      if t in ['(','['] and t in tmp:
        tmp.remove(t)
        for f in tokens[n:]:
          if f in [')',']'] and f in tmp:
            tmp.remove(f)
            break
          if f in tmp:
            tmp.remove(f)
      elif len(t)<=2 and t in tmp  and t not in stop:
        tmp.remove(t)  

    string = " ".join(tmp)
    words = re.findall(r'\b[A-Za-z]+\b',string)

    for word in words:
      fwrite.write(word+' ')
    fwrite.write('\n\n')

fwrite.close()

# **Create Own Word Embeddings**

### Download and load pre trained glove embeddings

In [9]:
cd tf_ner/data/example/

/content/tf_ner/data/example


In [10]:
!make download-glove
!make build

wget -P . "http://nlp.stanford.edu/data/glove.840B.300d.zip"
--2018-12-04 08:22:34--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2018-12-04 08:22:34--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip]
Saving to: ‘./glove.840B.300d.zip’

glove.840B.300d.zip 100%[===================>]   2.03G  27.9MB/s    in 53s     

2018-12-04 08:23:26 (39.4 MB/s) - ‘./glove.840B.300d.zip’ saved [2176768927/2176768927]

unzip glove.840B.300d.zip -d glove.840B.300d.txt
Archive:  glove.840B.300d.zip
  inflating: glove.840B.300d.txt/glove.840B.300d.txt  
rm glove.

###Fine tune glove embeddings with domain specifiic data

In [0]:
def glove2dict(glove_filename):
    with open(glove_filename) as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed

In [0]:
original_embedding = glove2dict("/content/tf_ner/data/example/glove.840B.300d.txt/glove.840B.300d.txt")

In [59]:
#mittens provides an interface for fine tuning word embeddings
!pip3 install mittens
from mittens import Mittens

In [0]:
testa = open('/content/drive/My Drive/NLP Project/testa.words.txt')
testb = open('/content/drive/My Drive/NLP Project/testb.words.txt')
train = open('/content/drive/My Drive/NLP Project/train/train.words.txt')
linesa = testa.readlines()
linesb = testb.readlines()
lines = train.readlines()
vocab_list = []
vb_list = []
for line in linesa:
  tokens = word_tokenize(line)
  vb_list.append(tokens)
  vocab_list = vocab_list + tokens 
for line in linesb:
  tokens = word_tokenize(line)
  vb_list.append(tokens)
  vocab_list = vocab_list + tokens 
for line in lines:
  tokens = word_tokenize(line)
  vb_list.append(tokens)
  vocab_list = vocab_list + tokens 
myset = set(vocab_list)
vocab = list(myset)

In [0]:
document = vb_list
names = vocab
array = [[0 for i in range(len(vocab))] for j in range(len(vocab))]

for l in document:
    for i in range(len(l)):
        for item in l[:i] + l[i + 1:]:
            array[i][vocab.index(item)] += 1

In [0]:
array = np.array(array)
cooccurrence = array

In [18]:
mittens_model = Mittens(n = 300, max_iter=1000)
new_embeddings = mittens_model.fit(cooccurrence, vocab = vocab, initial_embedding_dict= original_embedding)

Iteration 1000: loss: 86.0621337890625

# **Tensorflow - Named Entity Recognition**

https://github.com/guillaumegenthial/tf_ner

In [42]:
cd /content/drive/My\ Drive/NLP\ Project/train

/content/drive/My Drive/NLP Project/train


In [41]:
!cp /content/drive/My\ Drive/NLP\ Project/train/train.words.txt .
!cp /content/drive/My\ Drive/NLP\ Project/train/train.tags.txt .

cp: cannot stat '/content/drive/My Drive/NLP Project/train/train.words.txt': No such file or directory
cp: cannot stat '/content/drive/My Drive/NLP Project/train/train.tags.txt': No such file or directory


### Create vocab and initialise word embeddings.

In [46]:
!python build_vocab.py

Build vocab words (may take a while)
- done. Kept 3789 out of 3789
Build vocab chars
- done. Found 26 chars
Build vocab tags (may take a while)
- done. Found 13 tags.


In [47]:
with Path('vocab.words.txt').open() as f:
  word_to_idx = {line.strip(): idx for idx, line in enumerate(f)}
  size_vocab = len(word_to_idx)

  # Array of zeros
embeddings = np.zeros((size_vocab, 300))
found = 0
for n,v in enumerate(vocab):
  word = v
  embedding = new_embeddings[n]
  if word in word_to_idx:
      found += 1
      word_idx = word_to_idx[word]
      embeddings[word_idx] = embedding
print('- done. Found {} vectors for {} words'.format(found, size_vocab))
np.savez_compressed('/content/drive/My Drive/NLP Project/train/glove.npz', embeddings=embeddings)

- done. Found 3789 vectors for 3789 words


### Move to the directory where your model is present

In [0]:
cd /content/tf_ner/models/chars_conv_lstm_crf_ema/

### Train the model

*   List item
*   List item



In [49]:
"""GloVe Embeddings + chars conv and max pooling + bi-LSTM + CRF"""

import functools
import json
import logging
from pathlib import Path
import sys

import numpy as np
import tensorflow as tf
from tf_metrics import precision, recall, f1

from masked_conv import masked_conv1d_and_max

DATADIR = '/content/drive/My Drive/NLP Project/train/'

# Logging
Path('results').mkdir(exist_ok=True)
tf.logging.set_verbosity(logging.INFO)
handlers = [
    logging.FileHandler('results/main.log'),
    logging.StreamHandler(sys.stdout)
]
logging.getLogger('tensorflow').handlers = handlers


def parse_fn(line_words, line_tags):
    # Encode in Bytes for TF
    words = [w.encode() for w in line_words.strip().split()]
    tags = [t.encode() for t in line_tags.strip().split()]
    assert len(words) == len(tags), "Words and tags lengths don't match"

    # Chars
    chars = [[c.encode() for c in w] for w in line_words.strip().split()]
    lengths = [len(c) for c in chars]
    max_len = max(lengths)
    chars = [c + [b'<pad>'] * (max_len - l) for c, l in zip(chars, lengths)]
    return ((words, len(words)), (chars, lengths)), tags


def generator_fn(words, tags):
    with Path(words).open('r') as f_words, Path(tags).open('r') as f_tags:
        for line_words, line_tags in zip(f_words, f_tags):
            yield parse_fn(line_words, line_tags)


def input_fn(words, tags, params=None, shuffle_and_repeat=False):
    params = params if params is not None else {}
    shapes = ((([None], ()),               # (words, nwords)
               ([None, None], [None])),    # (chars, nchars)
              [None])                      # tags
    types = (((tf.string, tf.int32),
              (tf.string, tf.int32)),
             tf.string)
    defaults = ((('<pad>', 0),
                 ('<pad>', 0)),
                'O')
    dataset = tf.data.Dataset.from_generator(
        functools.partial(generator_fn, words, tags),
        output_shapes=shapes, output_types=types)

    if shuffle_and_repeat:
        dataset = dataset.shuffle(params['buffer']).repeat(params['epochs'])

    dataset = (dataset
               .padded_batch(params.get('batch_size', 20), shapes, defaults)
               .prefetch(1))
    return dataset


def graph_fn(features, labels, mode, params, reuse=None, getter=None):
    # Read vocabs and inputs
    num_tags = params['num_tags']
    (words, nwords), (chars, nchars) = features
    training = (mode == tf.estimator.ModeKeys.TRAIN)
    with tf.variable_scope('graph', reuse=reuse, custom_getter=getter):
        # Read vocabs and inputs
        dropout = params['dropout']
        (words, nwords), (chars, nchars) = features
        training = (mode == tf.estimator.ModeKeys.TRAIN)
        vocab_words = tf.contrib.lookup.index_table_from_file(
            params['words'], num_oov_buckets=params['num_oov_buckets'])
        vocab_chars = tf.contrib.lookup.index_table_from_file(
            params['chars'], num_oov_buckets=params['num_oov_buckets'])
        with Path(params['tags']).open() as f:
            indices = [idx for idx, tag in enumerate(f) if tag.strip() != 'O']
            num_tags = len(indices) + 1
        with Path(params['chars']).open() as f:
            num_chars = sum(1 for _ in f) + params['num_oov_buckets']

        # Char Embeddings
        char_ids = vocab_chars.lookup(chars)
        variable = tf.get_variable(
            'chars_embeddings', [num_chars + 1, params['dim_chars']], tf.float32)
        char_embeddings = tf.nn.embedding_lookup(variable, char_ids)
        char_embeddings = tf.layers.dropout(char_embeddings, rate=dropout,
                                            training=training)

        # Char LSTM
        weights = tf.sequence_mask(nchars)
        char_embeddings = masked_conv1d_and_max(
            char_embeddings, weights, params['filters'], params['kernel_size'])

        # Word Embeddings
        word_ids = vocab_words.lookup(words)
        glove = np.load(params['glove'])['embeddings']  # np.array
        variable = np.vstack([glove, [[0.] * params['dim']]])
        variable = tf.Variable(variable, dtype=tf.float32, trainable=False)
        word_embeddings = tf.nn.embedding_lookup(variable, word_ids)

        # Concatenate Word and Char Embeddings
        embeddings = tf.concat([word_embeddings, char_embeddings], axis=-1)
        embeddings = tf.layers.dropout(embeddings, rate=dropout, training=training)

        # LSTM
        t = tf.transpose(embeddings, perm=[1, 0, 2])  # Need time-major
        lstm_cell_fw = tf.contrib.rnn.LSTMBlockFusedCell(params['lstm_size'])
        lstm_cell_bw = tf.contrib.rnn.LSTMBlockFusedCell(params['lstm_size'])
        lstm_cell_bw = tf.contrib.rnn.TimeReversedFusedRNN(lstm_cell_bw)
        output_fw, _ = lstm_cell_fw(t, dtype=tf.float32, sequence_length=nwords)
        output_bw, _ = lstm_cell_bw(t, dtype=tf.float32, sequence_length=nwords)
        output = tf.concat([output_fw, output_bw], axis=-1)
        output = tf.transpose(output, perm=[1, 0, 2])
        output = tf.layers.dropout(output, rate=dropout, training=training)

        # CRF
        logits = tf.layers.dense(output, num_tags)
        crf_params = tf.get_variable("crf", [num_tags, num_tags], dtype=tf.float32)
        return logits, crf_params


def ema_getter(ema):

    def _ema_getter(getter, name, *args, **kwargs):
        var = getter(name, *args, **kwargs)
        ema_var = ema.average(var)
        return ema_var if ema_var else var

    return _ema_getter


def model_fn(features, labels, mode, params):
    # For serving features are a bit different
    if isinstance(features, dict):
        features = ((features['words'], features['nwords']),
                    (features['chars'], features['nchars']))

    with Path(params['tags']).open() as f:
        indices = [idx for idx, tag in enumerate(f) if tag.strip() != 'O']
        num_tags = len(indices) + 1
        params['num_tags'] = num_tags

    # Graph
    (words, nwords), (chars, nchars) = features
    logits, crf_params = graph_fn(features, labels, mode, params)
    pred_ids, _ = tf.contrib.crf.crf_decode(logits, crf_params, nwords)

    # Moving Average
    variables = tf.get_collection('trainable_variables', 'graph')
    ema = tf.train.ExponentialMovingAverage(0.999)
    ema_op = ema.apply(variables)
    logits_ema, crf_params_ema = graph_fn(
        features, labels, mode, params, reuse=True, getter=ema_getter(ema))
    pred_ids_ema, _ = tf.contrib.crf.crf_decode(
        logits_ema, crf_params_ema, nwords)

    if mode == tf.estimator.ModeKeys.PREDICT:
        # Predictions
        reverse_vocab_tags = tf.contrib.lookup.index_to_string_table_from_file(
            params['tags'])
        pred_strings = reverse_vocab_tags.lookup(tf.to_int64(pred_ids))
        pred_strings_ema = reverse_vocab_tags.lookup(tf.to_int64(pred_ids_ema))
        predictions = {
            'pred_ids': pred_ids,
            'tags': pred_strings,
            'pred_ids_ema': pred_ids_ema,
            'tags_ema': pred_strings_ema,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    else:
        # Loss
        vocab_tags = tf.contrib.lookup.index_table_from_file(params['tags'])
        tags = vocab_tags.lookup(labels)
        log_likelihood, _ = tf.contrib.crf.crf_log_likelihood(
            logits, tags, nwords, crf_params)
        loss = tf.reduce_mean(-log_likelihood)

        # Metrics
        weights = tf.sequence_mask(nwords)
        metrics = {
            'acc': tf.metrics.accuracy(tags, pred_ids, weights),
            'acc_ema': tf.metrics.accuracy(tags, pred_ids_ema, weights),
            'pr': precision(tags, pred_ids, num_tags, indices, weights),
            'pr_ema': precision(tags, pred_ids_ema, num_tags, indices, weights),
            'rc': recall(tags, pred_ids, num_tags, indices, weights),
            'rc_ema': recall(tags, pred_ids_ema, num_tags, indices, weights),
            'f1': f1(tags, pred_ids, num_tags, indices, weights),
            'f1_ema': f1(tags, pred_ids_ema, num_tags, indices, weights),
        }
        for metric_name, op in metrics.items():
            tf.summary.scalar(metric_name, op[1])

        if mode == tf.estimator.ModeKeys.EVAL:
            return tf.estimator.EstimatorSpec(
                mode, loss=loss, eval_metric_ops=metrics)

        elif mode == tf.estimator.ModeKeys.TRAIN:
            train_op = tf.train.AdamOptimizer().minimize(
                loss, global_step=tf.train.get_or_create_global_step(),
                var_list=variables)
            train_op = tf.group([train_op, ema_op])
            return tf.estimator.EstimatorSpec(
                mode, loss=loss, train_op=train_op)


if __name__ == '__main__':
    # Params
    params = {
        'dim_chars': 100,
        'dim': 300,
        'dropout': 0.5,
        'num_oov_buckets': 1,
        'epochs': 25,
        'batch_size': 5,
        'buffer': 15000,
        'filters': 50,
        'kernel_size': 3,
        'lstm_size': 100,
        'words': str(Path(DATADIR, 'vocab.words.txt')),
        'chars': str(Path(DATADIR, 'vocab.chars.txt')),
        'tags': str(Path(DATADIR, 'vocab.tags.txt')),
        'glove': str(Path(DATADIR, 'glove.npz'))
    }
    with Path('results/params.json').open('w') as f:
        json.dump(params, f, indent=4, sort_keys=True)

    def fwords(name):
        return str(Path(DATADIR, '{}.words.txt'.format(name)))

    def ftags(name):
        return str(Path(DATADIR, '{}.tags.txt'.format(name)))

    # Estimator, train and evaluate
    train_inpf = functools.partial(input_fn, fwords('train'), ftags('train'),
                                   params, shuffle_and_repeat=True)
    eval_inpf = functools.partial(input_fn, fwords('testa'), ftags('testa'))

    cfg = tf.estimator.RunConfig(save_checkpoints_secs=120)
    estimator = tf.estimator.Estimator(model_fn, 'results/model', cfg, params)
    Path(estimator.eval_dir()).mkdir(parents=True, exist_ok=True)
    hook = tf.contrib.estimator.stop_if_no_increase_hook(
        estimator, 'f1_ema', 500, min_steps=8000, run_every_secs=120)
    train_spec = tf.estimator.TrainSpec(input_fn=train_inpf, hooks=[hook])
    eval_spec = tf.estimator.EvalSpec(input_fn=eval_inpf, throttle_secs=120)
    print("SSUP22",train_spec,eval_spec)
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    print("OH")
    # Write predictions to file
    def write_predictions(name, mode):
        """Write predictions of dataset with name to file"""
        Path('results/score').mkdir(parents=True, exist_ok=True)
        with Path('results/score/{}.{}.preds.txt'.format(name, mode)).open('wb') as f:
            test_inpf = functools.partial(input_fn, fwords(name), ftags(name))
            golds_gen = generator_fn(fwords(name), ftags(name))
            preds_gen = estimator.predict(test_inpf)
            for golds, preds in zip(golds_gen, preds_gen):
                ((words, _), (_, _)), tags = golds
                for word, tag, tag_pred in zip(words, tags, preds[mode]):
                    f.write(b' '.join([word, tag, tag_pred]) + b'\n')
                f.write(b'\n')

    for name in ['train', 'testa', 'testb']:
        for mode in ['tags', 'tags_ema']:
          write_predictions(name, mode)

Using config: {'_model_dir': 'results/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 120, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f26d3bbbf98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
SSUP22 TrainSpec(input_fn=functools.partial(<function input_fn at 0x7f26d3b71598>, '/content/drive/My Drive/NLP Project/train/train.words.txt', '/content/drive/My Drive/NLP Project/train/train.tags.

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Done calling model_fn.
Create CheckpointSaverHook.
Graph was finalized.
Running local_init_op.
Done running local_init_op.
Saving checkpoints for 0 into results/model/model.ckpt.
loss = 373.88907, step = 1
global_step/sec: 0.90124
loss = 47.537987, step = 101 (110.960 sec)
Saving checkpoints for 106 into results/model/model.ckpt.
Calling model_fn.
Done calling model_fn.
Starting evaluation at 2018-12-04-09:17:15
Graph was finalized.
Restoring parameters from results/model/model.ckpt-106
Running local_init_op.
Done running local_init_op.
Finished evaluation at 2018-12-04-09:17:18
Saving dict for global step 106: acc = 0.8873469, acc_ema = 0.8253061, f1 = 0.13218391, f1_ema = 0.025157232, global_step = 106, loss = 106.509056, pr = 0.37704918, pr_ema = 0.031578947, rc = 0.080139376, rc_ema = 0.020905923
Saving 'checkpoint_path' summary for global step 106: results/model/model.ckpt-106
Saving checkpoints for 184 into results/model/model.ckpt.
Calling model_fn.
Done calling model_fn.
Starti

In [0]:
!cp -r results /content/drive/My\ Drive/NLP\ Project/train

In [53]:
ls results/score

testa.tags_ema.preds.txt  testb.tags_ema.preds.txt  train.tags_ema.preds.txt
testa.tags.preds.txt      testb.tags.preds.txt      train.tags.preds.txt
